In [39]:
# GOALS
# 2. Create a scoreing based off views that will score a thumbnail and likes
 # this score could be more accurate if watch time was incorporated
# 3. create a basic predicter that can predict a videos outcome based on features I choose

# using the information I get make some video recommendations 
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array

D:\Anaconda\envs\learn-env\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
# This cell reads in the data I scraped from youtube

df = pd.read_csv(r'C:\Users\jared\data_science\youtube\mr_beast_analysis\mr_beast_channel_analysis\Video_data.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               150 non-null    object 
 1   description         150 non-null    object 
 2   videoLength         150 non-null    int64  
 3   thumbnailUrl        150 non-null    object 
 4   videoId             150 non-null    object 
 5   videoUrl            150 non-null    object 
 6   channelId           150 non-null    object 
 7   channelUrl          150 non-null    object 
 8   keywords            11 non-null     object 
 9   averageRating       0 non-null      float64
 10  author              150 non-null    object 
 11  profileUrl          150 non-null    object 
 12  viewCount           150 non-null    int64  
 13  likeCount           150 non-null    int64  
 14  dislikeCount        0 non-null      float64
 15  authorThumbnailUrl  150 non-null    object 
 16  subscrib

,title,description,videoLength,thumbnailUrl,videoId,videoUrl,channelId,channelUrl,keywords,averageRating,...,likeCount,dislikeCount,authorThumbnailUrl,subscriberCount,category,publishDate,isLiveContent,isFamilySafe,query,timestamp
0,"Extreme $1,000,000 Hide And Seek",I didn't expect that to happen at the end I wa...,729,https://i.ytimg.com/vi/00NgUctWoLQ/hqdefault.jpg,00NgUctWoLQ,https://www.youtube.com/watch?v=00NgUctWoLQ,UCX6OQ3DkcsbYNE6H8uQQuVA,https://www.youtube.com/channel/UCX6OQ3DkcsbYN...,NaN,NaN,...,2429798,NaN,https://yt3.ggpht.com/ytc/AKedOLSJtY3pQz27cdDo...,87.1M subscribers,Entertainment,2021-12-18,False,True,"https://www.youtube.com/watch?v=00NgUctWoLQ',",2022-01-02T04:55:06.903Z
1,"$1,000,000 Influencer Tournament!",That was so close at the end I can't believe t...,724,https://i.ytimg.com/vi/x9TQ6culXIA/hqdefault.jpg,x9TQ6culXIA,https://www.youtube.com/watch?v=x9TQ6culXIA,UCX6OQ3DkcsbYNE6H8uQQuVA,https://www.youtube.com/channel/UCX6OQ3DkcsbYN...,NaN,NaN,...,2404541,NaN,https://yt3.ggpht.com/ytc/AKedOLSJtY3pQz27cdDo...,87.1M subscribers,Entertainment,2021-12-16,False,True,"https://www.youtube.com/watch?v=x9TQ6culXIA',",2022-01-02T04:55:07.894Z
2,I Got Hunted By The Military,I'm not sure how they drove over that... it wa...,661,https://i.ytimg.com/vi/QbJJwaVdgIs/maxresdefau...,QbJJwaVdgIs,https://www.youtube.com/watch?v=QbJJwaVdgIs,UCX6OQ3DkcsbYNE6H8uQQuVA,https://www.youtube.com/channel/UCX6OQ3DkcsbYN...,NaN,NaN,...,2481993,NaN,https://yt3.ggpht.com/ytc/AKedOLSJtY3pQz27cdDo...,87.1M subscribers,Entertainment,2021-12-10,False,True,"https://www.youtube.com/watch?v=QbJJwaVdgIs',",2022-01-02T04:55:08.752Z
3,"$456,000 Squid Game In Real Life!",MAKE SURE YOU WATCH UNTIL GLASS BRIDGE IT'S IN...,1541,https://i.ytimg.com/vi/0e3GPea1Tyg/maxresdefau...,0e3GPea1Tyg,https://www.youtube.com/watch?v=0e3GPea1Tyg,UCX6OQ3DkcsbYNE6H8uQQuVA,https://www.youtube.com/channel/UCX6OQ3DkcsbYN...,NaN,NaN,...,12616243,NaN,https://yt3.ggpht.com/ytc/AKedOLSJtY3pQz27cdDo...,87.1M subscribers,Entertainment,2021-11-24,False,True,"https://www.youtube.com/watch?v=0e3GPea1Tyg',",2022-01-02T04:55:09.420Z
4,I Cleaned The World’s Dirtiest Beach #TeamSeas,$1 Donated = 1 Pound Of Trash Out Of The Ocean...,295,https://i.ytimg.com/vi/cV2gBU6hKfY/maxresdefau...,cV2gBU6hKfY,https://www.youtube.com/watch?v=cV2gBU6hKfY,UCX6OQ3DkcsbYNE6H8uQQuVA,https://www.youtube.com/channel/UCX6OQ3DkcsbYN...,NaN,NaN,...,4162031,NaN,https://yt3.ggpht.com/ytc/AKedOLSJtY3pQz27cdDo...,87.1M subscribers,Entertainment,2021-10-29,False,True,"https://www.youtube.com/watch?v=cV2gBU6hKfY',",2022-01-02T04:55:10.117Z


In [4]:
# In this cell I make additional features I thought could be useful to check if they have an impact on views

df['description_length'] = [len(x) for x in list(df['description'])]
df['title_length'] = [len(x) for x in list(df['title'])]

In [5]:
df['description_length']

0       374
1       438
2      1237
3       631
4       156
       ... 
145     479
146     811
147     464
148     403
149     501
Name: description_length, Length: 150, dtype: int64

In [6]:
# Dropping some unneeded features, I checked each for useability beforehand
df = df.drop('thumbnailUrl',axis=1)
df = df.drop('videoUrl',axis=1)
df = df.drop('channelId',axis=1)
df = df.drop('channelUrl',axis=1)
df = df.drop('keywords',axis=1)
df = df.drop('averageRating',axis=1)
df = df.drop('dislikeCount',axis=1)
df = df.drop('subscriberCount',axis=1)
df = df.drop('author',axis=1)
df = df.drop('profileUrl',axis=1)
df = df.drop('authorThumbnailUrl',axis=1)
df = df.drop('timestamp',axis=1)
df = df.drop('query',axis=1)
df = df.drop('isFamilySafe',axis=1)
df = df.drop('category',axis=1)
df = df.drop(df.index[df['videoId'] == '_uwNDiU04zE'],axis=0)
df = df.drop(df.index[df['videoId'] == 'YSoJPA8-oHc'],axis=0)

In [7]:
df.head(10)

,title,description,videoLength,videoId,viewCount,likeCount,publishDate,isLiveContent,description_length,title_length
0,"Extreme $1,000,000 Hide And Seek",I didn't expect that to happen at the end I wa...,729,00NgUctWoLQ,41202575,2429798,2021-12-18,False,374,32
1,"$1,000,000 Influencer Tournament!",That was so close at the end I can't believe t...,724,x9TQ6culXIA,38442440,2404541,2021-12-16,False,438,33
2,I Got Hunted By The Military,I'm not sure how they drove over that... it wa...,661,QbJJwaVdgIs,46288104,2481993,2021-12-10,False,1237,28
3,"$456,000 Squid Game In Real Life!",MAKE SURE YOU WATCH UNTIL GLASS BRIDGE IT'S IN...,1541,0e3GPea1Tyg,192952648,12616243,2021-11-24,False,631,33
4,I Cleaned The World’s Dirtiest Beach #TeamSeas,$1 Donated = 1 Pound Of Trash Out Of The Ocean...,295,cV2gBU6hKfY,53352795,4162031,2021-10-29,False,156,46
5,"Last To Take Hand Off Lamborghini, Keeps It",Subscribe if you want to compete!\nWatch My Do...,942,qIsgdOVGA04,55510031,2698382,2021-10-23,False,568,43
6,"First To Rob Bank Wins $100,000",I didnt think he would actually rob the bank.....,482,cExLQ1o2pDw,52272016,2397995,2021-09-26,False,660,31
7,"Extreme $500,000 Game Of Tag!",I did not expect that to happen at the end...\...,760,uocETPj4Jx4,59219952,2802519,2021-09-04,False,507,29
8,"Last To Leave Circle Wins $500,000",THIS WAS THE CRAZIEST THING IVE EVER DONE!\nGe...,1165,zxYjTTXc-J8,101193933,3556109,2021-08-31,False,804,34
9,I Got Hunted By The FBI,You guys loved the bounty hunter video so we d...,1000,hxwpkM5w3Cc,73456912,2582514,2021-08-07,False,753,23


In [33]:
# Checking out Titles compared to views top 25

df_title_and_views = df[['title','viewCount','title_length']]
df_title_and_views.sort_values('viewCount', ascending=False, inplace=True)
df_title_and_views.head(25)

<ipython-input-33-39acf1240205>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_title_and_views.sort_values('viewCount', ascending=False, inplace=True)


,title,viewCount,title_length
3,"$456,000 Squid Game In Real Life!",192952648,33
50,I Spent 50 Hours In Solitary Confinement,160357867,40
19,I Spent 50 Hours Buried Alive,156104424,29
144,I Put 100 Million Orbeez In My Friend's Backyard,152117417,48
30,"Press This Button To Win $100,000!",148004153,34
59,Anything You Can Fit In The Circle I’ll Pay For,146628344,47
84,"Going Through The Same Drive Thru 1,000 Times",139555608,45
60,"I Ate A $70,000 Golden Pizza",128667086,28
41,"Last To Leave $800,000 Island Keeps It",121190544,38
125,I Went Back To 1st Grade For A Day,120500264,34


In [34]:
# Comparing the bottom 25 now
df_title_and_views.tail(25)

,title,viewCount,title_length
39,"Get This Random Person 1,000,000 Subscribers",41949412,44
80,"I Spent $100,000 On Lottery Tickets And Won!",41358878,44
0,"Extreme $1,000,000 Hide And Seek",41202575,32
145,"Spending $40,000 In One Hour Challenge",40589449,38
124,I Advertised Pewdiepie At The Super Bowl,39509112,40
71,"Last To Stop Riding Bike Wins $1,000,000 (Part 4)",38624348,49
1,"$1,000,000 Influencer Tournament!",38442440,33
143,Destroyed Friend's House And Bought Him A New One,38297758,49
126,I Opened The World’s Cheapest Store,37372259,35
97,"Can 50,000 Magnets Catch A Cannon Ball?",36631325,39


In [35]:
# Seeing if title lenght has an impact
df_title_and_views.corr()

,viewCount,title_length
viewCount,1.000000,0.029162
title_length,0.029162,1.000000


# Notes on Video Titles
- My basic hypothesis going into this was that videos containing explicit numbers would perform better than abstract videos. When looking at the results this isn't really the case. The top 25 performing videos have 20 videos with either a large sum of money in the title or a large amount of hours in the title. The bottom 25 performing videos have 19 videos with a large sum of money or a large amount of hours in the title. With a one video difference I think that's enough to disprove my hypothesis. This data shows it's possible to create a good title without including these large numbers. I think it also enforces the idea that the Mr. Beast team is very talented with naming videos. The reason I say this is because when examining the video titles they correlate with the channel's age. In the top 25 the majority of the videos are from last year (2021), and the bottom 25 contains a lot of videos from three years ago (2018). The key takeaway from looking at the titles is that the method the Mr. Beast team uses to name videos works, and they should stick with it. I also thought it could be useful to check if a video's title length had any impact on performance and the data gave me a resounding no. The length of a video title has no real impact on a video's performance.

In [27]:
# Now looking at video lenght
df_length_and_views = df[['videoLength','viewCount']]
df_length_and_views.sort_values('viewCount', ascending=False, inplace=True)
df_length_and_views.head(25)

<ipython-input-27-9b6c79f10d82>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_length_and_views.sort_values('viewCount', ascending=False, inplace=True)


,videoLength,viewCount
3,1541,192952648
50,951,160357867
19,759,156104424
144,885,152117417
30,911,148004153
59,1199,146628344
84,937,139555608
60,775,128667086
41,1335,121190544
125,1065,120500264


In [29]:
df_length_and_views.tail(25)

,videoLength,viewCount
39,844,41949412
80,912,41358878
0,729,41202575
145,842,40589449
124,764,39509112
71,705,38624348
1,724,38442440
143,967,38297758
126,919,37372259
97,913,36631325


In [32]:
# Applying Numeric Calculation for a Clear Result

df_length_and_views.corr()

,videoLength,viewCount
videoLength,1.000000,-0.119207
viewCount,-0.119207,1.000000


# Examining Video Length and Views
- When looking at the numbers in the top and bottom twenty-five it's hard to make any sense of it. The column for video length is in seconds which only makes it more confusing to understand. Thankfully using the .corr() python method I'm able to apply statistical calculations on the data set and get a much clearer numeric result called a r score. Based on the result I again come to the conclusion that the team at Mr. Beast has already nailed video length. The r score I got indicates a slight negative relationship between Video Length and views. However the score is so low and very close to zero that it's not worth worrying about. The r score would need to be much closer to 1 or -1 to be worrying. A score this close to zero could easily just be from noise in the data. In order to further explore this relationship it could be worthwhile to decrease the next two or three videos total length and then proceed to release two or three videos back at the original video length. You could then run the numbers again to get a new r score with a much clearer result on the relationship between the length of a Mr. Beast video and it's overall performance

In [77]:
# Now I want to look at thumbnails, I want to build a model that will give a thumbnail a score based on view count and likes

veiws_and_likes = pd.DataFrame(df[['videoId','viewCount','likeCount']])

directory = "thumbnails/"

thumbnails_dict = {}
# This code is importing the thumbnails, and then saving the image array with the youtube video ID
for filename in os.listdir(directory):
    image = Image.open("C:/Users/jared/data_science/youtube/mr_beast_analysis/mr_beast_channel_analysis/thumbnails/"+ filename).resize((500,620))
    image = img_to_array(image).flatten()
    thumbnails_dict[filename] = image

print(thumbnails_dict)

{'00NgUctWoLQ.jpg': array([ 73.,  68.,  62., ..., 224., 221., 198.], dtype=float32), '0DaMUhgcAqo.jpg': array([ 68.,  82.,  49., ..., 192., 187., 184.], dtype=float32), '0e3GPea1Tyg.jpg': array([128., 228., 254., ...,   7.,  39.,  36.], dtype=float32), '0hVZOJCYBBM.jpg': array([143., 116.,  95., ...,  89.,  65.,  42.], dtype=float32), '0NGWT9COcEI.jpg': array([ 77.,  71.,  71., ..., 172., 175., 180.], dtype=float32), '1SwVSTKFCsc.jpg': array([215.,  60.,   1., ..., 140., 131., 116.], dtype=float32), '1V_03efSkg0.jpg': array([ 31., 188., 229., ...,  32., 188., 229.], dtype=float32), '2eFSU7TFOnk.jpg': array([193., 218., 248., ..., 180., 191., 197.], dtype=float32), '2nd73lyvq4w.jpg': array([40., 38., 43., ..., 62., 57., 64.], dtype=float32), '3ix_gbcubTo.jpg': array([199., 219., 228., ...,   1., 220., 156.], dtype=float32), '3LLnjRLrvLU.jpg': array([180., 181., 140., ...,  11.,   9.,  10.], dtype=float32), '3TflpIllQHY.jpg': array([101.,  89.,  96., ..., 135., 108.,  85.], dtype=float32

In [80]:
thumbs = pd.DataFrame(thumbnails_dict, index=['image', 'videoId'])
thumbs.head()

ValueError: Shape of passed values is (930000, 148), indices imply (2, 148)

In [ ]:
veiws_and_likes.head()

In [192]:
# Adding the photos to the data frame
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img

train_datagen = ImageDataGenerator()
Thumbnails = train_datagen.flow_from_directory(r'C:\Users\jared\data_science\youtube\mr_beast_analysis\mr_beast_channel_analysis',save_format='jpg')
print(Thumbnails)

df['thumbnails'] = 0

Found 148 images belonging to 3 classes.


In [ ]:
thumbs

In [196]:
df.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
           dtype='int64', length=148)

0